# Markov Chains
Markov Chain is a very important concept in the field of machine learning. It is defined as a random process that transitions from one state to another in the state space. That is, at each step of the Markov chain, the system can choose to change their state to a different one or remain the same according to a probability distribution. A change in state is called a transition, and the probabilities associated with different state changes are called transition probabilities. The probability distribution of the process depends only on the current state, not the events preceding it. This is known as the 'memoryless property' as well as the 'Markov property'. With this property, the modeling process of many problems is greatly simplified as it reduces complex dependencies to only one state.

<img 
    style="display: block; 
           margin-left: auto;
           margin-right: auto;
           width: 50%;"
    src="../Examples/Markov_Chain.png" 
    alt="markov_chain">
</img>

## Implementation

We can start our implementation by viewing an agent's gameplay as a Markov chain. Import `numpy` library, and define the transition probability of weather.

In [19]:
import numpy as np
import pandas as pd

# agent can take three actions: go to left, go to right and fire
agent_states = ["left", "right", "fire"]
transition_matrix = [
    [0.7, 0.1, 0.2], 
    [0.4, 0.2, 0.4], 
    [0.3, 0.5, 0.2]
]
df = pd.DataFrame(transition_matrix, index=agent_states, columns=agent_states)
print(f"Transition probability: \n{df}")

Transition probability: 
       left  right  fire
left    0.7    0.1   0.2
right   0.4    0.2   0.4
fire    0.3    0.5   0.2


Then we simulate the state transition process of Markov chain.

In [20]:
# use dictionary to make the process more readable
def convert_to_dict(states, matrix):
    transition_dict = {}
    for i, state in enumerate(states):
        transition_dict[state] = {states[j]: matrix[i][j] for j in range(len(states))}
    return transition_dict

transition_prob = convert_to_dict(agent_states, transition_matrix)
print(transition_prob)

def state_trans(cur_state, agent_states, transition_prob):
    state = np.random.choice(agent_states, p=[transition_prob[cur_state][next_state] for next_state in agent_states])
    return state
    
cur_state = "left"
print(f"Original state: {cur_state}")
step = 10
for i in range(step):
    next_state = state_trans(cur_state, agent_states, transition_prob)
    print(f"Current state: {next_state}")
    cur_state = next_state

{'left': {'left': 0.7, 'right': 0.1, 'fire': 0.2}, 'right': {'left': 0.4, 'right': 0.2, 'fire': 0.4}, 'fire': {'left': 0.3, 'right': 0.5, 'fire': 0.2}}
Original state: left
Current state: left
Current state: fire
Current state: fire
Current state: right
Current state: left
Current state: left
Current state: left
Current state: left
Current state: fire
Current state: right


Let's try a more complex example on a Super Mario Bros level. First, read our prepared SMB level from "/emamples" file. A Markov chain model is constructed based on these level data. The code generates a Markov chain representing the probability of state transitions by recording how often a particular tile appears in different states. In this notebook, we only used one level as training data, more level can be added to this process to obtain a better result. The original code can be found at: [SMB_Markov](https://github.com/PCGML-Book/Mariolike-Markov-Level-Generation/tree/main)

In [7]:
level = {}

with open('../examples/Mario.txt', 'r') as f:
		y = 0
		for line in f:
			level[y] = line.strip()
			y+=1

print(level)

# Extract Markov chain Counts from 'level'
markovCounts = {} # Dictionary of (x-1, y), (x-1, y+1), (x, y+1)

maxY = len(level)-1
for y in range(maxY, -1, -1):
	for x in range(0, len(level[y])-1):

		# This grabs the tile values to the left (west), below (south), and left and below (southwest)
		west = " "
		southwest = " "
		south = " "

		if x>0: 
			west = level[y][x-1]
		if y<maxY: 
			south = level[y+1][x-1]
		if x>0 and y<maxY: 
			southwest = level[y+1][x]

		state = west+southwest+south

		if not state in markovCounts.keys():
			markovCounts[state] = {}
		if not level[y][x] in markovCounts[state].keys():
			markovCounts[state][level[y][x]] = 0

		# Increments the number of times we see the tile value at location (x,y) given the state (the tile values at (x-1, y), (x-1, y+1), (x, y+1))
		markovCounts[state][level[y][x]] +=1.0

# Normalize markov counts in order to approximate probability values
markovProbabilities = {} # The representation of our Markov chain, a dictionary of dictionaries 
for state in markovCounts.keys():
	markovProbabilities[state] = {}

	sumVal = 0
	for action in markovCounts[state].keys():
		sumVal+=markovCounts[state][action]
	for action in markovCounts[state].keys():
		markovProbabilities[state][action] =markovCounts[state][action]/sumVal # Approximation of probability values of seeing tile value 'action' given the current 'state'

print(markovProbabilities)

{0: '----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------', 1: '----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------', 2: '----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------', 3: '----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------', 4: '---------------------------------------------------------------------------------------------------------------------------------------------------------------

Create a new SMB level based on the previously trained Markov chain model, generate a new game level and write it to a txt file.

In [14]:
import random
new_level = {}

# Parameters determining the size of the new
maxY = 15 # will end up generating a new level with one height larger than this
maxX = 100

# Starting in the bottom left corner, we begin the generation process, going bottom to top then left to right
for y in range(maxY, -1, -1):
	new_level[y] =""
	for x in range(0, maxX): # We generate one tile at a time for each iteration of this inner loop

		# Grab the current state, the three dependent values
		west = " "
		southwest = " "
		south = " "

		if x>0: 
			west = new_level[y][x-1]
		if y<maxY: 
			south = new_level[y+1][x-1]
		if x>0 and y<maxY: 
			southwest = new_level[y+1][x]

		state = west+southwest+south

		# Query the Markov chain to see what tile value we should place at this tile location
		if state in markovProbabilities.keys():
			
			# Greedy Sampling. 
			# Uncomment this and comment the Weighted Sampling section below to see what greedy sampling looks like (and why we don't tend to use it)
			'''
			maxValueTile = "-"
			maxValue = 0.0
			for action in markovProbabilities[state]:
				if markovProbabilities[state][action] >maxValue:
					maxValue = markovProbabilities[state][action]
					maxValueTile = action
			new_level[y] +=maxValueTile #Add the tile value (tokenToUse) to the level
			'''
			# Weighted Sampling
			randValue = random.random()
			currProb = 0
			tokenToUse = "-"
			for action in markovProbabilities[state]:
				currProb+=markovProbabilities[state][action]
				if currProb>randValue:
					tokenToUse = action
					break
			
			new_level[y] += tokenToUse # Add the tile value (tokenToUse) to the level
			
		else:
			# If we can't find anything, just output an empty space
			new_level[y] +="-"

f = open("../examples/New_Level.txt","w")
for y in range(0, maxY+1):
	f.write(new_level[y]+"\n")
f.close()

## Best practices
- Not all problems can be represented as a markov chain, such as fluctuations in stoke market prices. Markov chain is a powerful method, but the applicable scenarios for your problems must be determined precisely.
- The definition of state space should cover as many possible states as possible, but not so complex as to raise the computation cost. Also, they should be clear enough and independent of each other.
- Ensure the sum of the transition probability matrix of each state to other states equals 1.